### Решение задач ДЗ


### 1.

1) Рассчитать "Активная сумма в нац. Валюте" и кол-во в разрезе "Подразделение"

2) Сделать градацию по суммам (до 10 000; 50 000; 200 000; 500 000; более 500 000) "Активная сумма в нац. Валюте"

3) Рассчитать "Активная сумма в нац. Валюте" в разрезе "Подсектор кредита", а также Рассчитать Портфель в риске (до 30 дней; 90 дней; 180 дней; более 180 дней) в разрезе "Подразделение"

In [6]:
# Import Modules
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_columns", 150)
pd.set_option("display.max_rows", 100)

In [7]:
# Read Data
df = pd.read_excel(r"C:\Users\EgamovA\Desktop\Courses\10Активные кредиты 31.10.20X6.xlsx", header=3)
# "C:\Users\EgamovA\Desktop\Courses\10Активные кредиты 31.10.20X6.xlsx"

In [48]:
# Manual actions
result = df.groupby('Подразделение').agg(active_sum_lcy=("Активная сумма в нац. валюте", "sum"),
                                        active_count_lcy=("Активная сумма в нац. валюте", "count")
                                        ).reset_index()

# Cross-check
if result['active_sum_lcy'].isna().any():
    raise Warning(f"Please check your calculations by department {result.loc[result['active_sum_lcy'].isna(), 'Подразделение'].unique().tolist()}")

In [46]:
result.loc[result.active_sum_lcy.eq(6454026070), 'active_sum_lcy'] = None

In [49]:
result

# Cross-check
if result['active_sum_lcy'].isna().any():
    raise Warning(f"Please check your calculations by department {result.loc[result['active_sum_lcy'].isna(), 'Подразделение'].unique().tolist()}")

#### 2.

In [59]:
def categorize_amount(amount):
    if amount <= 10000:
        return 'до 10 000'
    elif amount <= 50000:
        return '10 000 - 50 000'
    elif amount <= 200000:
        return '50 000 - 200 000'
    elif amount <= 500000:
        return '200 000 - 500 000'
    else:
        return 'более 500 000'

# Создание нового столбца с категориями
df['category'] = df['Активная сумма в нац. валюте'].apply(categorize_amount)

if not df.loc[df['category'].isna()].empty:
    raise Warning(f"Some data isn't categorized. \n Please check this clients {df.loc[df['category'].isna(), 'Рег. номер клиента'].unique().tolist()}")

#### 3

In [71]:
sector = df.groupby(['Подсектор (Вид деятельности)', 'Филиал'])['Активная сумма в нац. валюте'].sum().reset_index()

def calculate_portfolio_risk(days):
    if days <= 30:
        return 'до 30 дней'
    elif days <= 90:
        return 'до 90 дней'
    elif days <= 180:
        return 'до 180 дней'
    else:
        return 'более 180 дней'

df['Портфель в риске'] = df['Количество дней просрочки'].apply(calculate_portfolio_risk)

risk_portfolio = df.groupby(['Подразделение', 'Портфель в риске'])['Активная сумма в нац. валюте'].sum().reset_index()

In [74]:
df['Дата выдачи'] = pd.to_datetime(df['Дата выдачи'], format='%d.%m.%Y')
df['Дата окончание действия договора'] = pd.to_datetime(df['Дата окончание действия договора'], format='%d.%m.%Y')

df['Количество дней для оплаты'] = (df['Дата окончание действия договора'] - df['Дата выдачи']).dt.days

### 

### Day 7. Reporting   

Сценарие: В прошлом месяце был объявлен конкурс лучший КЭ со следующими критериями:
1. Прирост кредитного портфеля на 30%
2. Доля клиентов женского пола в портфеле не менее 20%
3. Доля просрочек 10 дней и более (PAR 10+)  меньше 1%
4. Наличие не менее 10 лояльных клиентов (ступень кредита >5)
5. Наличие проектных кредитов
   
Первое место с максимальными показателями получить 5000 сомони , 2-е место - 3000 и 3-е 1000 сом.

Определите 3 победилей конкурса, анализируя данные текущего месяца и прошлого.


Сценарие: При построении модели для скоринга кредитов, группа аналитиков столкнулась с проблемой не качественного ввода данных.
Требуется анализировать средные показатели по следующим полям:
1. Ежемесячный доход семьи	
2. Ежемесячный расход семьи	
3. Дата рождения клиента

Следует учесть фактор региона/филиала и пол
